In [ ]:
import pandas as pd
import folium
from folium import plugins
import json
import geopandas as gpd

In [ ]:
### [[수원시]] _ GeoJson
## MultiLineString
suwon_road = json.load(open('data/geojson/28.수원시_도로명주소(도로).geojson', encoding='utf-8'))

##MultiPolygon
#행정동
suwon_Area = json.load(open('data/geojson/31.수원시_행정경계(읍면동).geojson', encoding='utf-8'))

suwon_building = json.load(open('data/geojson/29.수원시_도로명주소(건물).geojson', encoding='utf-8'))

In [ ]:
suwon_Time_Pop = pd.read_csv('data/csv/4.수원시_시간대별_유동인구(2020).csv', encoding='utf-8')  #
suwon_SexAge_Pop = pd.read_csv('data/csv/5.수원시_성연령별_유동인구(2020).csv', encoding='utf-8')  #
suwon_Day_Pop = pd.read_csv('data/csv/6.수원시_요일별_유동인구(2020).csv', encoding='utf-8')  #

## 성연령별 유동인구

In [ ]:
display(suwon_SexAge_Pop.head())
print(len(suwon_SexAge_Pop))
display(suwon_SexAge_Pop.describe())

In [ ]:
suwon_SexAge_Pop.columns = ['년월', '남10', '남20', '남30', '남40', '남50', '남60', '여10', '여20', '여30', '여40', '여50', '여60', 'lon', 'lat']

In [ ]:
def normalize(df, c_name) :
    df[c_name] = (df[c_name] - min(df[c_name])) /  (max(df[c_name]) - min(df[c_name]))

def make_matrix(df, empty_matrix, c_name) :
    ls = []
    temp = df[df['년월']==int(i)]
    for lat, lon, cnt in zip(temp['lat'], temp['lon'], temp[c_name]) :
        ls.append([lat, lon, cnt])
    empty_matrix.append(ls)
    
def change_date(x) :
    x = str(x)
    return x[:4]+'-'+x[4:6]+'-'+x[6:]

In [ ]:
Sex_Pop =  pd.DataFrame({'년월': suwon_SexAge_Pop['년월'],
                        '남자' : suwon_SexAge_Pop['남10']+ suwon_SexAge_Pop['남20']+suwon_SexAge_Pop['남30']+suwon_SexAge_Pop['남40']+suwon_SexAge_Pop['남50']+suwon_SexAge_Pop['남60'],
                        '여자' : suwon_SexAge_Pop['여10']+ suwon_SexAge_Pop['여20']+suwon_SexAge_Pop['여30']+suwon_SexAge_Pop['여40']+suwon_SexAge_Pop['여50']+suwon_SexAge_Pop['여60'],
                        'lon' : suwon_SexAge_Pop['lon'],
                        'lat' : suwon_SexAge_Pop['lat']})
Age_Pop = pd.DataFrame({'년월': suwon_SexAge_Pop['년월'],
                        '10대' : suwon_SexAge_Pop['남10']+ suwon_SexAge_Pop['여10'],
                        '20대' : suwon_SexAge_Pop['남20']+ suwon_SexAge_Pop['여20'],
                        '30대' : suwon_SexAge_Pop['남30']+ suwon_SexAge_Pop['여30'],
                        '40대' : suwon_SexAge_Pop['남40']+ suwon_SexAge_Pop['여40'],
                        '50대' : suwon_SexAge_Pop['남50']+ suwon_SexAge_Pop['여50'],
                        '60대' : suwon_SexAge_Pop['남60']+ suwon_SexAge_Pop['여60'],
                        'lon' : suwon_SexAge_Pop['lon'],
                        'lat' : suwon_SexAge_Pop['lat']})

In [ ]:
Sex_index = ['남자', '여자']
Age_index = ['10대', '20대', '30대', '40대', '50대', '60대']

In [ ]:
for n in Sex_index :
    normalize(Sex_Pop, n)
    
for n in Age_index :
    normalize(Age_Pop, n)

In [ ]:
display(Sex_Pop.head())
display(Age_Pop.head())
date = list(suwon_SexAge_Pop['년월'].unique())
m_matrix = []
w_matrix = []
age10_matrix = []
age20_matrix = []
age30_matrix = []
age40_matrix = []
age50_matrix = []
age60_matrix = []

In [ ]:
display(Sex_Pop[Sex_Pop['년월']==202001])

In [ ]:
ex_matrix = [m_matrix, w_matrix]
Age_matrix = [age10_matrix, age20_matrix, age30_matrix, age40_matrix, age50_matrix, age60_matrix]
for m, index in zip(Sex_matrix, Sex_index) :
    for i in date : 
        make_matrix(Sex_Pop, m, index)
    
for m, index in zip(Age_matrix, Age_index) :
    for i in date : 
        make_matrix(Age_Pop, m, index)
        
time_index = list(map(lambda i : change_date(i), date))

In [ ]:
def visualize(matrix, index) : 
    center = [37.280364359333866, 127.00776116595848] # 수원 가운데

    map = folium.Map(location=center,
                   tiles='cartodbpositron',
                   zoom_start=12)

    dong_area_map = folium.Choropleth(
                        geo_data = suwon_Area,
                        line_weight = 1,
                        line_color = 'black',
                        line_opacity = 1,
                        fill_opacity = 0, 
                        highlight = True,
                        name = '행정동 경계',
                    ).add_to(map)
    dong_area_map.geojson.add_child(
        folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
    )

    plugins.HeatMapWithTime(
    data = matrix,
    index = time_index,
    radius = 12,
    name = f'2020년도_월별_{index}_유동인구',
    auto_play = True,

    ).add_to(map)

    #folium.LayerControl().add_to(suwon_map)
    map.save(f'2020년도_월별_{index}_유동인구.html')

In [ ]:
for m, i in zip(Sex_matrix, Sex_index) :
    visualize(m,i)

In [ ]:
for m, i in zip(Age_matrix, Age_index) :
    visualize(m,i)

## 요일별 유동인구

In [ ]:
display(suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_MON','lon','lat']][suwon_Day_Pop['STD_YM']==202001])
print( (suwon_Day_Pop['FLOW_POP_CNT_MON']- min(suwon_Day_Pop['FLOW_POP_CNT_MON'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_MON']) - min(suwon_Day_Pop['FLOW_POP_CNT_MON'])))
#print(suwon_Day_Pop['FLOW_POP_CNT_MON'].sum()/len)

date = list(suwon_Day_Pop['STD_YM'].unique())

mon_matrix = []
tue_matrix = []
wed_matrix = []
thu_matrix = []
fri_matrix = []
sat_matrix = []
sun_matrix = []

#정규화
suwon_Day_Pop['FLOW_POP_CNT_MON'] = (suwon_Day_Pop['FLOW_POP_CNT_MON'] - min(suwon_Day_Pop['FLOW_POP_CNT_MON'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_MON']) - min(suwon_Day_Pop['FLOW_POP_CNT_MON']))
suwon_Day_Pop['FLOW_POP_CNT_TUS'] = (suwon_Day_Pop['FLOW_POP_CNT_TUS'] - min(suwon_Day_Pop['FLOW_POP_CNT_TUS'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_TUS']) - min(suwon_Day_Pop['FLOW_POP_CNT_TUS']))
suwon_Day_Pop['FLOW_POP_CNT_WED'] = (suwon_Day_Pop['FLOW_POP_CNT_WED'] - min(suwon_Day_Pop['FLOW_POP_CNT_WED'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_WED']) - min(suwon_Day_Pop['FLOW_POP_CNT_WED']))
suwon_Day_Pop['FLOW_POP_CNT_THU'] = (suwon_Day_Pop['FLOW_POP_CNT_THU'] - min(suwon_Day_Pop['FLOW_POP_CNT_THU'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_THU']) - min(suwon_Day_Pop['FLOW_POP_CNT_THU']))
suwon_Day_Pop['FLOW_POP_CNT_FRI'] = (suwon_Day_Pop['FLOW_POP_CNT_FRI'] - min(suwon_Day_Pop['FLOW_POP_CNT_FRI'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_FRI']) - min(suwon_Day_Pop['FLOW_POP_CNT_FRI']))
suwon_Day_Pop['FLOW_POP_CNT_SAT'] = (suwon_Day_Pop['FLOW_POP_CNT_SAT'] - min(suwon_Day_Pop['FLOW_POP_CNT_SAT'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_SAT']) - min(suwon_Day_Pop['FLOW_POP_CNT_SAT']))
suwon_Day_Pop['FLOW_POP_CNT_SUN'] = (suwon_Day_Pop['FLOW_POP_CNT_SUN'] - min(suwon_Day_Pop['FLOW_POP_CNT_SUN'])) /  (max(suwon_Day_Pop['FLOW_POP_CNT_SUN']) - min(suwon_Day_Pop['FLOW_POP_CNT_SUN']))

In [ ]:
for i in date : 
    mon = []
    mon_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_MON','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(mon_temp['lat'], mon_temp['lon'], mon_temp['FLOW_POP_CNT_MON']) :
        mon.append([lat, lon, cnt])
    mon_matrix.append(mon)

for i in date : 
    tue = []
    tue_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_TUS','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(tue_temp['lat'], tue_temp['lon'], tue_temp['FLOW_POP_CNT_TUS']) :
        tue.append([lat, lon, cnt])
    tue_matrix.append(tue)
    

for i in date :     
    wed = []
    wed_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_WED','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(wed_temp['lat'], wed_temp['lon'], wed_temp['FLOW_POP_CNT_WED']) :
        wed.append([lat, lon, cnt])
    wed_matrix.append(wed)
    

for i in date :     
    thu = []
    thu_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_THU','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(thu_temp['lat'], thu_temp['lon'], thu_temp['FLOW_POP_CNT_THU']) :
        thu.append([lat, lon, cnt])
    thu_matrix.append(thu)
    
for i in date :     
    fri = []
    fri_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_FRI','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(fri_temp['lat'], fri_temp['lon'], fri_temp['FLOW_POP_CNT_FRI']) :
        fri.append([lat, lon, cnt])
    fri_matrix.append(fri)
    
for i in date :     
    sat = []
    sat_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_SAT','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(sat_temp['lat'], sat_temp['lon'], sat_temp['FLOW_POP_CNT_SAT']) :
        sat.append([lat, lon, cnt])
    sat_matrix.append(sat)
    
for i in date :     
    sun = []
    sun_temp = suwon_Day_Pop[['STD_YM','FLOW_POP_CNT_SUN','lon','lat']][suwon_Day_Pop['STD_YM']==i]
    for lat, lon, cnt in zip(sun_temp['lat'], sun_temp['lon'], sun_temp['FLOW_POP_CNT_SUN']) :
        sun.append([lat, lon, cnt])
    sun_matrix.append(sun)
    
# print(len(date))
# print(len(mon_matrix))
# print(mon_matrix[0])


time_index = list(map(lambda i : change_date(i), date))

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_mon = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
#matrix = [mon_matrix, tue_matrix, wed_matrix, thu_matrix, fri_matrix, sat_matrix, sun_matrix]

dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_mon)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

#yoil = ['월','화','수','목','금','토','일']


    
# for문으로 다객체 생성 시도 / timeslider 중복 문제 발생
#for m, y in zip(matrix, yoil) :

plugins.HeatMapWithTime(
        data = mon_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_월요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_mon)

#folium.LayerControl().add_to(suwon_map)
suwon_mon.save('2020년도_월별_월요일_유동인구.html')

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_tue = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_tue)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

plugins.HeatMapWithTime(
        data = tue_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_화요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_tue)
suwon_tue.save('2020년도_월별_화요일_유동인구.html')

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_wed = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_wed)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

plugins.HeatMapWithTime(
        data = wed_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_수요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_wed)
suwon_wed.save('2020년도_월별_수요일_유동인구.html')

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_thu = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_thu)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

plugins.HeatMapWithTime(
        data = thu_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_목요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_thu)
suwon_thu.save('2020년도_월별_목요일_유동인구.html')

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_fri = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_fri)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

plugins.HeatMapWithTime(
        data = fri_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_금요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_fri)
suwon_fri.save('2020년도_월별_금요일_유동인구.html')

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_sat = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_sat)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

plugins.HeatMapWithTime(
        data = sat_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_토요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_sat)
suwon_sat.save('2020년도_월별_토요일_유동인구.html')

In [ ]:
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_sun = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)
dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 1,
                    line_color = 'black',
                    line_opacity = 1,
                    fill_opacity = 0, 
                    highlight = True,
                    name = '행정동 경계',
                ).add_to(suwon_sun)
dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)

plugins.HeatMapWithTime(
        data = sun_matrix,
        index = time_index,
        radius = 12,
        name = '2020년도_월별_일요일_유동인구',
        auto_play = True,
        
    ).add_to(suwon_sun)
suwon_sun.save('2020년도_월별_일요일_유동인구.html')